In [1]:
import os
from pathlib import Path

TUTORIAL_DIR = Path(os.getcwd()).as_posix()

# Energytool Modifier

In a renovation process, choosing between Energy Efficiency Measure (EEM) is a difficult tasks. There is often a lot of measures, and they can be combined (insulate walls + change boiler), (insulate walls + change windows), (change windows + change boiler).
Also several indicator are used to make a choice :
- EEM cost
- EEM effect on energy savings
- EEM effect on occupant thermal comfort
- EEM CO2 cost
- ...

The objective of the <code>modifier</code> module is to provide a framework to specify all the possible EEM. Simulate every possible combination, and provide metrics to the user to guide him in his choice.

## Building definition

In this tutorial we will use the same building as the one describe in the Building tutorial. Please refer to this file to understand <code>Building</code> and hvac system modeling.

The use case is a 2 floors 4 apartments residential building.

In [46]:
from energytool.building import Building
from energytool.indicators import AddOutputVariables
import energytool.system as st

In [47]:
building = Building(idf_path=Path(TUTORIAL_DIR) / "resources/tuto_building.idf")

In [48]:
# Simulate a boiler, multiplying the heat needs by a constant COP
building.heating_system["Main_boiler"] = st.HeaterSimple(
    name="Gaz_boiler",
    building=building,
    cop=0.89
)

# Estimate circulation pumps energy consumption multiplying the heat needs by a constant (default 0.05)
building.heating_system["Circulation_pumps"] = st.AuxiliarySimplified(
    name="Heater_aux",
    building=building
)

# Simulate fan consumption multiplying extracted air volume by a constant coefficient
# Do not have a heat exchanger
building.ventilation_system["Main_AHU"] = st.AirHandlingUnit(
    name="Extraction_fan",
    building=building,
    fan_energy_coefficient=0.23, # Wh/m3
    heat_recovery_efficiency=False
)

# Simulate clock regulation
# Ventilation works according to specified schedule
#  is defined in the energytool/resources/resources_idf.idf file
building.ventilation_system["AHU_control"] = st.AHUControl(
    name="Hygro_intakes",
    building=building,
    control_strategy="Schedule",
    schedule_name="OFF_09h_18h_ON_18h_24h_ON_WE_FULL_YEAR",
)

# Estimate Domestic Hot Water production energy needs
# Use the number of people defined in the idf file to estimate the total volume.
# Otherwise, energy calculation is independent of energyplus
building.dwh_system["DHW_production"] = st.DHWIdealExternal(
    name="Electric_accumulation",
    building=building,
    cop=0.85,
)

# Estimate Domestic Hot Water production energy needs
# Use the number of people defined in the idf file to estimate the total volume.
# Otherwise, energy calculation is independent of energyplus
building.dwh_system["DHW_production"] = st.DHWIdealExternal(
    name="Electric_accumulation",
    building=building,
    cop=0.85,
)

# Estimate Lighting consumption using a constant power ratio.
# Modify the existing energyplus object
building.artificial_lighting_system["Lights"] = st.ArtificialLightingSimple(
    name="Random_lights",
    building=building,
    power_ratio=4 # W/m²
)

# Add 2 variables for summer thermal comfort calculation
building.other["Thermal_comfort_variables"] = AddOutputVariables(
    name="HQE_discomfort",
    building=building,
    variables=["Zone Operative Temperature", "Zone People Occupant Count"]
)

## EEM specifications

In this tutorial, to reduce computation time, we will define 4 EEM and 3 modifiers :
- _EEM1_Wall_int_insulation_ : 15mm of glass wool on the inside of the building
- _EEM2_Wall_ext_insulation_ : 30mm of glass wool on the external face
- _EEM3_Double_glazing_ : Replace outside windows
- _EEM4_HP_main_heater_ : Replace existing heater by a heatpump

Let's initialize a modifier <code>list</code> where we will append all our modifications

In [70]:
modifier_list = []

## External wall EEM

External surface modification are done the same way using a python <code>dict</code>. The <code>keys</code> are the EEMs names, the <code>values</code> are a list of <code>dict</code> containing wall materials thermal properties.

The following example configure 2 EEMs named "EEM1_Wall_int_insulation" and "EEM2_Wall_ext_insulation". The first EEM have a 15cm glass wool internal insulation, the second have a 30cm glass wool external insulation

In [71]:
import energytool.modifier as mo

In [72]:
wall_variant_dict = {
    "EEM1_Wall_int_insulation": [
        # Outside Layer
        {
            "Name": "Project medium concrete block_.2",
            "Thickness": 0.2,
            "Conductivity": 0.51,
            "Density": 1400,
            "Specific_Heat": 1000,
        },
        {
            "Name": "Laine_15cm",
            "Thickness": 0.15,
            "Conductivity": 0.032,
            "Density": 40,
            "Specific_Heat": 1000,
        },
    ],
    "EEM2_Wall_ext_insulation": [
        # Outside Layer
        {
            "Name": "Coating",
            "Thickness": 0.01,
            "Conductivity": 0.1,
            "Density": 400,
            "Specific_Heat": 1200,
        },
        {
            "Name": "Laine_30cm",
            "Thickness": 0.30,
            "Conductivity": 0.032,
            "Density": 40,
            "Specific_Heat": 1000,
        },
        {
            "Name": "Project medium concrete block_.2",
            "Thickness": 0.2,
            "Conductivity": 0.51,
            "Density": 1400,
            "Specific_Heat": 1000,
        },
    ]
}

In [73]:
modifier_list.append(mo.OpaqueSurfaceModifier(
    name="ext_wall_mod",
    building=building,
    surface_type="Wall",
    outside_boundary_condition="Outdoors",
    variant_dict=wall_variant_dict
))

In the above code, we append to the <code>modifier_list</code> an <code>OpaqueSurfaceModifier</code> object.

This class is designed to replace the surface referenced by energyplus _surface_type_ and _outside_boundary_condition_ by one of the constructions defined in the <code>wall_variant_dict</code>.

Note that a "modifier" is NOT and EEM. In this case it holds 2 EEM (internal 15cm insulation and external 30cm insulation).

A modifier is designed to replace a part of the building model : replace wall, windows, infiltrations, hvac system, etc..

## External windows EEM

Similarly, we define EEM as a <code>dict</code>. <code>keys</code> are EEM names and <code>values</code> are dict defining energyplus object _WindowMaterial:SimpleGlazingSystem_

In [76]:
ext_win_variant_dict = {
    "EEM3_Double_glazing": {
        "Name": "Double_glazing",
        "UFactor": 1.1,
        "Solar_Heat_Gain_Coefficient": 0.41,
        "Visible_Transmittance": 0.71,
    },
}

In [77]:
modifier_list.append(mo.ExternalWindowsModifier(
    name="Window_modifier",
    building=building,
    variant_dict=ext_win_variant_dict
))

## HVAC EEM
HVAC EEM definition is a bit different. We first have to define a new <code>energytool.system</code>. Than we instantiate a <code>SystemModifier</code>, specifying the <code>Building</code> dictionary and the <code>key</code> holding the system that needs to be replaced.

For example, to replace the "Main_boiler":

In [78]:
boiler_variant_dict = {
    "EEM4_HP_main_heater": st.HeaterSimple(
        name="PAC", building=building, cop=3)
}

In [79]:
modifier_list.append(mo.SystemModifier(
    name="heater_modifier",
    building=building,
    category="heating_system",
    system_name="Main_boiler",
    variant_dict=boiler_variant_dict
))

## Combine EEM and run simulations

In order to combine all the EEM and to simulate their effect, we need to pass the previously defined list of modifiers to a <code>Combiner</code>

In [80]:
combiner = mo.Combiner(building, modifier_list=modifier_list)

The combinations can be found using THE property <code>combination_list</code>

In [81]:
combiner.combination_list

[('Existing', 'Existing', 'Existing'),
 ('Existing', 'Existing', 'EEM4_HP_main_heater'),
 ('Existing', 'EEM3_Double_glazing', 'Existing'),
 ('Existing', 'EEM3_Double_glazing', 'EEM4_HP_main_heater'),
 ('EEM1_Wall_int_insulation', 'Existing', 'Existing'),
 ('EEM1_Wall_int_insulation', 'Existing', 'EEM4_HP_main_heater'),
 ('EEM1_Wall_int_insulation', 'EEM3_Double_glazing', 'Existing'),
 ('EEM1_Wall_int_insulation', 'EEM3_Double_glazing', 'EEM4_HP_main_heater'),
 ('EEM2_Wall_ext_insulation', 'Existing', 'Existing'),
 ('EEM2_Wall_ext_insulation', 'Existing', 'EEM4_HP_main_heater'),
 ('EEM2_Wall_ext_insulation', 'EEM3_Double_glazing', 'Existing'),
 ('EEM2_Wall_ext_insulation', 'EEM3_Double_glazing', 'EEM4_HP_main_heater')]

Note that the combiner takes into account the Existing state when combining the EEMs. In fact the 1st simulation.
Running the simulations to get the sample is easy

In [82]:
combiner.run(
    epw_file_path=Path(TUTORIAL_DIR) / "resources/FRA_Bordeaux.075100_IWEC.epw",
    timestep_per_hour=1,
    nb_simu_per_batch=5
)

The <code>Combiner</code> method <code>get_annual_system_results</code> simplify the results formatting and gives annual results on hvac system consumption.
For now, summer thermal comfort has to be computed manually.

In [90]:
res = combiner.get_annual_system_results(per_square_meter=True)

In [92]:
res

,ext_wall_mod,Window_modifier,heater_modifier,Heating,Cooling,Ventilation,Lighting,DHW,Local_production,Total
0,Existing,Existing,Existing,44.030458,0.0,2.186344,24.388,24.137987,0.0,94.742789
1,Existing,Existing,EEM4_HP_main_heater,14.381737,0.0,2.186344,24.388,24.137987,0.0,65.094068
2,Existing,EEM3_Double_glazing,Existing,40.476353,0.0,2.186344,24.388,24.137987,0.0,91.188683
3,Existing,EEM3_Double_glazing,EEM4_HP_main_heater,13.220854,0.0,2.186344,24.388,24.137987,0.0,63.933185
4,EEM1_Wall_int_insulation,Existing,Existing,11.995722,0.0,2.186344,24.388,24.137987,0.0,62.708052
5,EEM1_Wall_int_insulation,Existing,EEM4_HP_main_heater,3.918181,0.0,2.186344,24.388,24.137987,0.0,54.630512
6,EEM1_Wall_int_insulation,EEM3_Double_glazing,Existing,9.421616,0.0,2.186344,24.388,24.137987,0.0,60.133947
7,EEM1_Wall_int_insulation,EEM3_Double_glazing,EEM4_HP_main_heater,3.077397,0.0,2.186344,24.388,24.137987,0.0,53.789728
8,EEM2_Wall_ext_insulation,Existing,Existing,8.705653,0.0,2.186344,24.388,24.137987,0.0,59.417984
9,EEM2_Wall_ext_insulation,Existing,EEM4_HP_main_heater,2.843541,0.0,2.186344,24.388,24.137987,0.0,53.555872


Sorted results ar useful to easily determine the best combination.
The <code>Combiner</code> method <code>plot_consumption_stacked_bar</code> gives a graphical representation of the results.

In [96]:
res.sort_values(by="Total", ascending=False)

,ext_wall_mod,Window_modifier,heater_modifier,Heating,Cooling,Ventilation,Lighting,DHW,Local_production,Total
0,Existing,Existing,Existing,44.030458,0.0,2.186344,24.388,24.137987,0.0,94.742789
2,Existing,EEM3_Double_glazing,Existing,40.476353,0.0,2.186344,24.388,24.137987,0.0,91.188683
1,Existing,Existing,EEM4_HP_main_heater,14.381737,0.0,2.186344,24.388,24.137987,0.0,65.094068
3,Existing,EEM3_Double_glazing,EEM4_HP_main_heater,13.220854,0.0,2.186344,24.388,24.137987,0.0,63.933185
4,EEM1_Wall_int_insulation,Existing,Existing,11.995722,0.0,2.186344,24.388,24.137987,0.0,62.708052
6,EEM1_Wall_int_insulation,EEM3_Double_glazing,Existing,9.421616,0.0,2.186344,24.388,24.137987,0.0,60.133947
8,EEM2_Wall_ext_insulation,Existing,Existing,8.705653,0.0,2.186344,24.388,24.137987,0.0,59.417984
10,EEM2_Wall_ext_insulation,EEM3_Double_glazing,Existing,6.510909,0.0,2.186344,24.388,24.137987,0.0,57.223240
5,EEM1_Wall_int_insulation,Existing,EEM4_HP_main_heater,3.918181,0.0,2.186344,24.388,24.137987,0.0,54.630512
7,EEM1_Wall_int_insulation,EEM3_Double_glazing,EEM4_HP_main_heater,3.077397,0.0,2.186344,24.388,24.137987,0.0,53.789728


In [97]:
combiner.plot_consumption_stacked_bar(per_square_meter=True)